# Setting optimal MLP's hiperparameters for news dataset

In [1]:
# Importing all necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import optuna

import pandas as pd
pd.set_option('display.max_columns', 70)

In [2]:
# Read preprocessed news dataset
news_df = pd.read_csv('./prep_news_data.csv')
news_df

,rank( timedelta),rank( n_tokens_title),rank( n_tokens_content),rank( n_unique_tokens),rank( n_non_stop_words),rank( n_non_stop_unique_tokens),rank( num_hrefs),rank( num_self_hrefs),num_imgs,num_videos,rank( average_token_length),rank( num_keywords),data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,rank( kw_max_min),rank( kw_avg_min),kw_min_max,kw_max_max,rank( kw_avg_max),kw_min_avg,rank( kw_max_avg),rank( kw_avg_avg),rank( self_reference_min_shares),rank( self_reference_max_shares),rank( self_reference_avg_sharess),weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,rank( LDA_00),rank( LDA_01),rank( LDA_02),rank( LDA_03),rank( LDA_04),rank( global_subjectivity),rank( global_sentiment_polarity),rank( global_rate_positive_words),rank( global_rate_negative_words),rank( rate_positive_words),rank( rate_negative_words),rank( avg_positive_polarity),min_positive_polarity,max_positive_polarity,rank( avg_negative_polarity),rank( min_negative_polarity),max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,3.091313,0.746312,-0.830751,1.171875,-0.859076,1.250329,-0.773700,-0.247319,1.0,0.0,0.060832,-1.077791,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-0.809729,-0.897243,-0.895353,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.011874,1.107189,0.003762,0.133906,-0.019697,0.834551,-0.299737,0.407371,-0.186943,0.425334,-0.344559,0.257404,0.100000,0.70,-0.840613,-0.309042,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,3.091313,-0.627749,-0.629945,0.636396,-0.682170,1.037852,-1.082209,-0.770293,1.0,0.0,0.874264,-1.654145,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-1.337803,-1.337726,-1.337803,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.506066,0.481113,0.164789,0.215657,0.071541,-1.206170,0.345029,0.255150,0.038674,0.167738,-0.092096,-0.900984,0.033333,0.70,1.303623,1.355812,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,3.091313,-0.627749,-0.877419,0.346405,-0.977397,-0.284000,-1.082209,-0.770293,1.0,0.0,-0.983430,-0.561454,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-0.314418,-0.753138,-0.722589,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.628018,-0.102896,-0.194219,-0.319837,1.102533,2.439542,2.017790,1.034661,-0.690612,1.106431,-0.977397,1.564511,0.100000,1.00,-1.637609,-0.868165,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,3.091313,-0.627749,0.309539,-0.344123,0.321434,-0.264207,0.207090,-1.494860,1.0,0.0,-0.947209,-0.086922,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-1.337803,-1.337726,-1.337803,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.437130,1.172315,0.875191,-0.392816,-0.811837,-0.291810,-0.208737,0.150161,0.574307,-0.284132,0.363172,0.351546,0.136364,0.80,-0.984764,-0.309042,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,3.091313,1.191898,1.253929,-1.200952,1.406604,-1.358432,1.046996,2.268357,20.0,0.0,0.069322,-0.086922,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-0.768720,1.144183,0.294747,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.314651,-0.220448,-0.531720,-0.626094,1.680026,0.738234,1.686806,1.934743,-0.367261,1.143818,-1.009924,0.671991,0.033333,1.00,0.327633,0.032505,-0.050000,0.454545,0.136364,0.045455,0.136364,505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39639,-3.716757,0.299307,-0.208059,-0.100354,-0.394045,-0.060357,0.207090,1.378593,1.0,1.0,-0.507837,0.343989,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.024409,-0.424676,26900.0,843300.0,0.91118

In [5]:
# Separate data into features and targets
X = news_df.iloc[:, :-1].values
y = news_df.iloc[:, -1].values
y = y.reshape(-1, 1)

# Data normalization
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)

# Splitting data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.1, random_state=42)

# Convertation data to PyTorch tensors
train_features = torch.tensor(X_train, dtype=torch.float32)
test_features = torch.tensor(X_test, dtype=torch.float32)
train_targets = torch.tensor(y_train, dtype=torch.float32)
test_targets = torch.tensor(y_test, dtype=torch.float32)    

# Creating DataLoader
train_data = TensorDataset(train_features, train_targets)
test_data = TensorDataset(test_features, test_targets)

train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

In [3]:
# Create a Neural Network class with flexible settings
class FlexibleRegressionNN(nn.Module):
    def __init__(self, input_size: int, hidden_layers_sizes: list, dropout_rate: float) -> None:
        super(FlexibleRegressionNN, self).__init__()
        layers = list()

        # Define first and n-1 hidden layers
        neuron_num = input_size
        for size in hidden_layers_sizes:
            layers.append(nn.Linear(neuron_num, size))
            layers.append(nn.BatchNorm1d(size))
            layers.append(nn.LeakyReLU())
            layers.append(nn.Dropout(dropout_rate))
            neuron_num = size

        layers.append(nn.Linear(neuron_num, 1))
        self.model = nn.Sequential(*layers)

    # Define forward loop for NN
    def forward(self, x):
        return self.model(x)

In [4]:
# Define function for calculation RMSE
def root_mean_squared_error(y_true, y_pred):
    mse = torch.mean((y_true - y_pred) ** 2)
    rmse = torch.sqrt(mse)
    return rmse.item()

In [7]:
# Creating function for training and evaluating the model
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        model.eval()
        with torch.no_grad():
            test_loss = 0.0
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
            test_loss /= len(test_loader)
        
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')
    
    return test_loss

# Creating function for selection of best hyperparameters 
def objective(trial):
    # Defining hyperparameters
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 20)
    hidden_layer_sizes = [
        trial.suggest_int(f'layer_{i}_size', 32, 256)
        for i in range(num_hidden_layers)
    ]
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    epochs = trial.suggest_int('epochs', 5, 100)
    input_size = 59

    # Creating a model and defining of optimizer
    model = FlexibleRegressionNN(input_size, hidden_layer_sizes, dropout_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    # Dataloader
    train_data = torch.randn(1000, input_size), torch.randn(1000, 1)
    test_data = torch.randn(200, input_size), torch.randn(200, 1)
    train_loader = DataLoader(TensorDataset(*train_data), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(*test_data), batch_size=batch_size, shuffle=False)

    # Training launch and return the test loss
    return train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs)

# Settings of searching by Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=230)

# Getting the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-12-26 11:43:42,946] A new study created in memory with name: no-name-c3a5546e-68a2-4888-b5ac-e084b0d1c851


/var/folders/qq/gt96f0sx7nn6wsr9d8n13r2w0000gn/T/ipykernel_83885/4155516338.py:36: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
/var/folders/qq/gt96f0sx7nn6wsr9d8n13r2w0000gn/T/ipykernel_83885/4155516338.py:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Epoch 1/88, Train Loss: 1.2208, Test Loss: 0.9766
Epoch 2/88, Train Loss: 1.1397, Test Loss: 0.9927
Epoch 3/88, Train Loss: 1.1520, Test Loss: 0.9977
Epoch 4/88, Train Loss: 1.1443, Test Loss: 1.0054
Epoch 5/88, Train Loss: 1.1566, Test Loss: 1.0008
Epoch 6/88, Train Loss: 1.1566, Test Loss: 0.9944
Epoch 7/88, Train Loss: 1.1496, Test Loss: 0.9934
Epoch 8/88, Train Loss: 1.1322, Test Loss: 0.9888
Epoch 9/88, Train Loss: 1.1313, Test Loss: 0.9815
Epoch 10/88, Train Loss: 1.1417, Test Loss: 0.9734
Epoch 11/88, Train Loss: 1.1015, Test Loss: 0.9768
Epoch 12/88, Train Loss: 1.1528, Test Loss: 0.9725
Epoch 13/88, Train Loss: 1.0913, Test Loss: 0.9687
Epoch 14/88, Train Loss: 1.0934, Test Loss: 0.9627
Epoch 15/88, Train Loss: 1.0836, Test Loss: 0.9561
Epoch 16/88, Train Loss: 1.1194, Test Loss: 0.9563
Epoch 17/88, Train Loss: 1.1267, Test Loss: 0.9598
Epoch 18/88, Train Loss: 1.1254, Test Loss: 0.9616
Epoch 19/88, Train Loss: 1.1204, Test Loss: 0.9602
Epoch 20/88, Train Loss: 1.1000, Test Lo

[I 2024-12-26 11:43:48,492] Trial 0 finished with value: 0.9587828516960144 and parameters: {'num_hidden_layers': 9, 'layer_0_size': 104, 'layer_1_size': 130, 'layer_2_size': 40, 'layer_3_size': 199, 'layer_4_size': 110, 'layer_5_size': 59, 'layer_6_size': 63, 'layer_7_size': 236, 'layer_8_size': 206, 'dropout_rate': 0.38039803353944734, 'learning_rate': 9.78130281307678e-05, 'batch_size': 128, 'epochs': 88}. Best is trial 0 with value: 0.9587828516960144.


Epoch 86/88, Train Loss: 1.0598, Test Loss: 0.9599
Epoch 87/88, Train Loss: 1.0577, Test Loss: 0.9610
Epoch 88/88, Train Loss: 1.0261, Test Loss: 0.9588
Epoch 1/33, Train Loss: 1.1686, Test Loss: 1.0060
Epoch 2/33, Train Loss: 1.0783, Test Loss: 1.0033
Epoch 3/33, Train Loss: 1.0418, Test Loss: 0.9984
Epoch 4/33, Train Loss: 1.0223, Test Loss: 0.9929
Epoch 5/33, Train Loss: 1.0099, Test Loss: 0.9920
Epoch 6/33, Train Loss: 0.9672, Test Loss: 1.0000
Epoch 7/33, Train Loss: 0.9477, Test Loss: 1.0141
Epoch 8/33, Train Loss: 0.9076, Test Loss: 1.0313
Epoch 9/33, Train Loss: 0.8814, Test Loss: 1.0475
Epoch 10/33, Train Loss: 0.8393, Test Loss: 1.0632
Epoch 11/33, Train Loss: 0.8089, Test Loss: 1.0809
Epoch 12/33, Train Loss: 0.8202, Test Loss: 1.0901
Epoch 13/33, Train Loss: 0.7747, Test Loss: 1.0997
Epoch 14/33, Train Loss: 0.7111, Test Loss: 1.1091
Epoch 15/33, Train Loss: 0.7346, Test Loss: 1.1266
Epoch 16/33, Train Loss: 0.6523, Test Loss: 1.1674
Epoch 17/33, Train Loss: 0.6162, Test Lo

...

Epoch 65/66, Train Loss: 0.9371, Test Loss: 1.0542
Epoch 66/66, Train Loss: 0.9136, Test Loss: 1.0513
Epoch 1/65, Train Loss: 1.1130, Test Loss: 1.1758
Epoch 2/65, Train Loss: 1.0939, Test Loss: 1.1821
Epoch 3/65, Train Loss: 1.1034, Test Loss: 1.2152
Epoch 4/65, Train Loss: 1.0422, Test Loss: 1.1812
Epoch 5/65, Train Loss: 1.0638, Test Loss: 1.1793
Epoch 6/65, Train Loss: 1.0518, Test Loss: 1.1632
Epoch 7/65, Train Loss: 1.0960, Test Loss: 1.1701
Epoch 8/65, Train Loss: 1.0048, Test Loss: 1.1736
Epoch 9/65, Train Loss: 1.0103, Test Loss: 1.1726
Epoch 10/65, Train Loss: 1.0636, Test Loss: 1.1764
Epoch 11/65, Train Loss: 1.0269, Test Loss: 1.1609
Epoch 12/65, Train Loss: 1.0025, Test Loss: 1.1853
Epoch 13/65, Train Loss: 1.0359, Test Loss: 1.1850
Epoch 14/65, Train Loss: 1.0538, Test Loss: 1.1785
Epoch 15/65, Train Loss: 1.0340, Test Loss: 1.1787
Epoch 16/65, Train Loss: 1.0271, Test Loss: 1.1740
Epoch 17/65, Train Loss: 0.9874, Test Loss: 1.1674
Epoch 18/65, Train Loss: 0.9969, Test Lo

[I 2024-12-26 12:14:53,701] Trial 228 finished with value: 1.2819907580103194 and parameters: {'num_hidden_layers': 9, 'layer_0_size': 229, 'layer_1_size': 111, 'layer_2_size': 195, 'layer_3_size': 121, 'layer_4_size': 90, 'layer_5_size': 136, 'layer_6_size': 225, 'layer_7_size': 187, 'layer_8_size': 72, 'dropout_rate': 0.256045803833059, 'learning_rate': 0.00035541461785542554, 'batch_size': 32, 'epochs': 65}. Best is trial 65 with value: 0.6746163879122052.


Epoch 64/65, Train Loss: 0.8164, Test Loss: 1.2945
Epoch 65/65, Train Loss: 0.8098, Test Loss: 1.2820
Epoch 1/63, Train Loss: 1.2506, Test Loss: 1.0491
Epoch 2/63, Train Loss: 1.1614, Test Loss: 1.0908
Epoch 3/63, Train Loss: 1.1630, Test Loss: 1.0468
Epoch 4/63, Train Loss: 1.1206, Test Loss: 1.0656
Epoch 5/63, Train Loss: 1.1860, Test Loss: 1.0667
Epoch 6/63, Train Loss: 1.1225, Test Loss: 1.0888
Epoch 7/63, Train Loss: 1.0939, Test Loss: 1.0512
Epoch 8/63, Train Loss: 1.0748, Test Loss: 1.0412
Epoch 9/63, Train Loss: 1.0558, Test Loss: 1.0530
Epoch 10/63, Train Loss: 1.0705, Test Loss: 1.0455
Epoch 11/63, Train Loss: 1.0195, Test Loss: 1.1118
Epoch 12/63, Train Loss: 1.0370, Test Loss: 1.0854
Epoch 13/63, Train Loss: 0.9796, Test Loss: 1.0801
Epoch 14/63, Train Loss: 0.9773, Test Loss: 1.1131
Epoch 15/63, Train Loss: 0.9778, Test Loss: 1.0849
Epoch 16/63, Train Loss: 0.9141, Test Loss: 1.0908
Epoch 17/63, Train Loss: 0.8888, Test Loss: 1.0675
Epoch 18/63, Train Loss: 0.8494, Test Lo

[I 2024-12-26 12:15:01,887] Trial 229 finished with value: 1.2306502205984933 and parameters: {'num_hidden_layers': 8, 'layer_0_size': 218, 'layer_1_size': 104, 'layer_2_size': 185, 'layer_3_size': 151, 'layer_4_size': 82, 'layer_5_size': 130, 'layer_6_size': 235, 'layer_7_size': 198, 'dropout_rate': 0.12886100853846166, 'learning_rate': 0.0005135808065726896, 'batch_size': 32, 'epochs': 63}. Best is trial 65 with value: 0.6746163879122052.


Epoch 63/63, Train Loss: 0.3448, Test Loss: 1.2307
Best trial:
  Value: 0.6746163879122052
  Params:
    num_hidden_layers: 12
    layer_0_size: 164
    layer_1_size: 191
    layer_2_size: 224
    layer_3_size: 114
    layer_4_size: 164
    layer_5_size: 90
    layer_6_size: 182
    layer_7_size: 208
    layer_8_size: 72
    layer_9_size: 43
    layer_10_size: 248
    layer_11_size: 76
    dropout_rate: 0.2542002795679332
    learning_rate: 0.00016799883676835427
    batch_size: 32
    epochs: 57
